[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/forestdatapartnership/whisp/blob/main/notebooks/Colab_whisp_geojson_to_csv.ipynb)

# Whisp a geojson

Python Notebook pathway for [Whisp](https://openforis.org/solutions/whisp/) running in the cloud via [Google Colab](https://colab.google/). 

**To open:** 
click badge at top. 

**To run:** click play buttons (or press shift + enter) 

**Requirements:** Google Earth Engine (GEE) account and registered cloud project.



- **Aim:** support compliance with zero deforestation regulations
- **Input**: geojson file of plot boundaries or points
- **Output**: CSV table and geojson containing statistics and risk indicators

### Setup Google Earth Engine

In [ ]:
import ee

# Google Earth Engine project name
gee_project_name = "your_cloud_project_name" # change to your project name. If unsure see here: https://developers.google.com/earth-engine/cloud/assets)

# NB opens browser to allow access
ee.Authenticate()

# initialize with chosen project
ee.Initialize(project=gee_project_name)

### Install and import packages

In [ ]:
# Install openforis-whisp (if not already installed)
!pip install --pre openforis-whisp

In [ ]:
import openforis_whisp as whisp

### Get a geojson

- Files are stored tempoarily and can be viewed in a panel on the left (click on Folder icon to view).
- Press refresh if updates are not showing
- Alternatively you can work with files in your Google Drive: drive.mount('/content/drive')

In [ ]:
#function to upload a geojson file. Download example here: https://github.com/forestdatapartnership/whisp/tree/main/tests/fixtures
def import_geojson():
    from google.colab import files
    fn, content = next(iter(files.upload().items()))
    with open(f'/content/{fn}', 'wb') as f: f.write(content)
    return f'/content/{fn}'

In [ ]:
GEOJSON_EXAMPLE_FILEPATH = import_geojson()
print(f"GEOJSON_EXAMPLE_FILEPATH: {GEOJSON_EXAMPLE_FILEPATH}")

### Prepare inputs

In [ ]:
# Choose countries to process (currently three countries: 'co', 'ci', 'br')
iso2_codes_list = ['co', 'ci', 'br']  # Example ISO2 codes for including country specific data

# Choose if want to include additional custom layers
USE_CUSTOM_BANDS = False # set to True if want to add extra ee data to whisp

In [ ]:
# =============================================================================
# CUSTOM BANDS SETUP (OPTIONAL) - runs only if USE_CUSTOM_BANDS = True above
# =============================================================================
if USE_CUSTOM_BANDS:
    
    # Step 1: Define custom Earth Engine images (binary values 0 or 1)
    custom_images = {
        'example_treecover': ee.Image(1),  
        'nBR_example_commodity': ee.Image.random(seed=1).gte(.5).reproject(crs='EPSG:4326', scale=10)
        # add more images as needed
    }
    
    # Step 2: Define metadata for each custom band (keys must match above)
    # Themes: 'treecover', 'commodities', 'disturbance_before', 'disturbance_after'
    # Timber themes: 'primary', 'naturally_reg_2020', 'planted_plantation_2020', etc.
    custom_bands_info = {
        'example_treecover': {
            'ISO2_code': "",          # Country code (empty = all countries)
            'theme': 'treecover',     # Risk theme
            'theme_timber': "",       # Timber theme (if applicable)
            'use_for_risk': 1,        # Include in risk calculations (1=yes, 0=no)
            'use_for_risk_timber': 0  # Include in timber risk (1=yes, 0=no)
        },
        'nBR_example_commodity': {
            'ISO2_code': "BR", 'theme': 'commodities', 'theme_timber': "", 
            'use_for_risk': 1, 'use_for_risk_timber': 0
        }
        # add more band metadata as needed
    }
    
    # Step 3: Combine custom bands and extract names
    custom_ee_image = whisp.combine_custom_bands(custom_images, custom_bands_info)
    
    custom_bands = list(custom_bands_info.keys())


In [ ]:
# Create final Whisp image
whisp_image = whisp.combine_datasets(national_codes=iso2_codes_list)

if USE_CUSTOM_BANDS and 'custom_ee_image' in locals():
    whisp_image = whisp_image.addBands(custom_ee_image)

print(f"Final image has {len(whisp_image.bandNames().getInfo())} bands")

### Run Whisp 

In [ ]:
df_stats = whisp.whisp_formatted_stats_geojson_to_df(
    input_geojson_filepath=GEOJSON_EXAMPLE_FILEPATH,
    # external_id_column="user_id", # optional -  specify which input column/property to map to the external ID.
    national_codes=iso2_codes_list,  # optional - By default national datasets are not included unless specified here.
    # unit_type='percent', # optional - to change unit type. Default is 'ha'. 
    whisp_image=whisp_image, # optional - defaults to standard whisp image if not provided
    custom_bands=custom_bands if USE_CUSTOM_BANDS else None  # include custom bands in formatted output 
) 

### Display results

In [ ]:
df_stats

### Add risk category columns

In [ ]:
# adds risk columns to end of dataframe
df_w_risk = whisp.whisp_risk(
    df=df_stats,
    national_codes=iso2_codes_list,
    custom_bands_info=custom_bands_info if USE_CUSTOM_BANDS else None  # Add: missing custom bands
)

### Display updated table
- Scroll to far right to see additions

In [ ]:
df_w_risk

### Export table with risk columns to CSV (temporary storage)

In [ ]:
df_w_risk.to_csv("whisp_output_table_w_risk.csv",index=False)

### Export table with risk columns to geojson (temporary storage)

In [ ]:
whisp.convert_df_to_geojson(df_w_risk,"whisp_output_table_w_risk.geojson") # builds a geojson file containing Whisp columns. Uses the geometry column "geo" to create the spatial features.

### Download outputs to local storage
- Saves files in "Downloads" folder on your machine
- If you see a "Downloads blocked" button at top of browser click to allow file downloads.
- Alternatively right click on file in the folder (in the panel on your left) and choose 'Download'.

In [ ]:
from google.colab import files
files.download('whisp_output_table_w_risk.csv')

In [ ]:
files.download('whisp_output_table_w_risk.geojson') # spatial output